In [ ]:
# default_exp pandas

# pandas

> In-memory processing

[Scaling suggestions](https://pandas.pydata.org/pandas-docs/stable/user_guide/scale.html) in official pandas documentation.

In [ ]:
data_dir = './out/extracts/100k'
data_years = range(1997, 2000)

# Repeated cross-section analysis

Every year can be processed one at a time.

In [ ]:
def comp_stats_like_a_pro(data, stats=None):
    """Flexible function to compute various summary statistics for given data.
    Data can be Series, DataFrame or either one groupped.
    Returns new DataFrame with stats in columns and variables and groups in rows."""

In [ ]:
fields = 'count count_na count_notna count_unique min max sum mean std p1 p25 p50 p75 p99'.split()
defaults = [pd.np.nan] * len(fields)
Stats = namedtuple('Stats', fields, defaults=defaults)

def comp_stats(s):
    """Return dictionary of stats computed from a Series."""
    count = len(s)
    na = s.isna().sum()
    if pd.api.types.is_numeric_dtype(s):
        q = s.quantile([0, 0.01, 0.25, 0.5, 0.75, 0.99, 1])
        num_stats = dict(min=q[0], max=q[1], sum=s.sum(), mean=s.mean(), std=s.std(),
                         p1=q[0.01], p25=q[0.25], p50=q[0.5], p75=q[0.75], p99=q[0.99])
    else:
        num_stats = dict()
    return Stats(count=count, count_na=na, count_notna=count - na, count_unique=s.nunique(), **num_stats)

In [ ]:
stats_by_year = {}
stat_cols = ['sales', 'employees', 'state', 'naics2']
for y in data_years:
    dt = dtypes_from_schema(schema_path)
    df = pd.read_csv(f'{data_dir}/{y}.csv', dtype=dt)
    df['naics2'] = df['naics'].str[:2]
    s = [comp_stats(df[c]) for c in stat_cols]
    s = pd.DataFrame(s, index=stat_cols)
    stats_by_year[y] = s
stats_by_year = pd.concat(stats_by_year).reorder_levels([1, 0]).sort_index()

In [ ]:
stats_by_year

In [ ]:
dfg = df.groupby('naics2')
dfg['sales'].apply(lambda c: pd.Series(comp_stats(c))).to_frame().unstack()

# Dynamic analysis

Growth rates that take two years simultaneously.

# Full panel analysis

Use all years to run a regression